# Generate graphs

In [1]:
import subprocess
import os
import numpy as np
import networkx as nx
import pandas as pd
import pickle
from scipy.sparse import csr_matrix,save_npz,load_npz


def mk_dir(export_dir, quite=False):
    if not os.path.exists(export_dir):
            try:
                os.makedirs(export_dir)
                print('created dir: ', export_dir)
            except OSError as exc: # Guard against race condition
                 if exc.errno != exc.errno.EEXIST:
                    raise
            except Exception:
                pass
    else:
        print('dir already exists: ', export_dir)


def generate_and_save_graphs(distribution,number_of_graphs, set_type, **kwargs):

    
    
    weighted=kwargs.get('weighted', True)
    n=kwargs.get('n', '')
    print(f'Creating {number_of_graphs} {distribution[1:]} graphs of {n} vertices for {set_type}')
    
    

    # check if all parameters are given 
    if distribution == '-rnd_graph':
        p = kwargs.get('p', None)
        assert p is not None,"Probability parameter is missing."
        print(f"Probablility : {p}")
        
    elif distribution == '-toroidal_grid_2D':
        heights = kwargs.get('heights', None)
        widths = kwargs.get('widths', None)
        n=heights[0]*widths[0]
    
        assert len(heights)==len(widths), 'Board sizes are missing'

        print(f"heights:",heights)
        print(f"widths:",widths)

    elif distribution == '-planar':
        density = kwargs.get('density', None)
        assert density is not None,"Density parameter is missing."
        print(f"density:",density)
    else:
        raise NotImplementedError('Unknown distribution.')

    # graphs = []
    if weighted:

        folder = f'data/{set_type}/{distribution[1:]}_{n}vertices_weighted'
    else:
        folder= f'data/{set_type}/{distribution[1:]}_{n}vertices_unweighted'
    mk_dir(folder)

    weighted=kwargs.get('weighted', True)

    for i in range(number_of_graphs):
        if distribution == '-rnd_graph':
            command = ["rudy", distribution, f"{n}", f"{p}", f'{i}']

        elif distribution == '-toroidal_grid_2D':
            idx = np.random.randint(len(heights))
            command = ["rudy", distribution, f"{heights[idx]}", f"{widths[idx]}"]
            print(command)

        elif distribution == '-planar':
            command = ["rudy", distribution, str(n), f"{density}", f'{i}',
                       distribution, str(n), str(density), f'{2*i+1}', '+']

        if weighted:
            # print('weighted graph')
            command += ["-random", '0', '1', f'{i}', '-times', '2', '-plus', '-1']

        save_path = f"{folder}/_temp"
        with open(save_path, "w") as output_file:
            subprocess.call(command, stdout=output_file)

        with open(save_path, "r") as output_file:
            content = output_file.read()

        os.remove(save_path)

        lines = content.split('\n')
        n = int(lines[0].split(' ')[0])
        graph = np.zeros(shape=(n, n))
        data = [tuple(map(int, line.split())) for line in lines[1:-1]]
        for u, v, w in data:
            graph[u-1, v-1] = w
            graph[v-1, u-1] = w

        # graph = nx.from_numpy_array(graph)
        # graphs.append(graph)

        # Save graphs
        if distribution == '-rnd_graph':
            save_file_name = f'{set_type}_rnd_{n}spins_p{p}_graph_{i}'
        elif distribution == '-toroidal_grid_2D':
            save_file_name = f'{set_type}_tor_{n}spins_graph_{i}'
        elif distribution == '-planar':
            save_file_name = f'{set_type}_planar_{n}spins_d{density}_graph_{i}'

        save_file_path=os.path.join(folder,save_file_name)
        
        sparse_matrix = csr_matrix(graph)
            
        save_npz(save_file_path, sparse_matrix)


# Random Graph

In [4]:
parameters={(800,6,False),(800,6,True),(2000,1,False),(2000,1,True),(1000,2,False)}

for n,p,weighted in parameters:
    generate_and_save_graphs(distribution='-rnd_graph', n=n, number_of_graphs=4000, set_type='training', 
                            p=p, weighted=weighted)
    generate_and_save_graphs(distribution='-rnd_graph', n=n, number_of_graphs=10, set_type='validation', 
                            p=p, weighted=weighted)
    

Creating 4000 rnd_graph graphs of 800 vertices for training
Probablility : 6
dir already exists:  data/training/rnd_graph_800vertices_weighted


FileNotFoundError: [Errno 2] No such file or directory: 'rudy.exe'

# Planar Graph

In [37]:
parameters={(800,99,False),(800,99,True),(2000,99,False),(2000,99,True),(1000,100,False)}

for n,density,weighted in parameters:
    generate_and_save_graphs(distribution='-planar', n=n, number_of_graphs=1, set_type='training', 
                            density=density, weighted=weighted)
    generate_and_save_graphs(distribution='-planar', n=n, number_of_graphs=1, set_type='validation', 
                            density=density, weighted=weighted)
    




Creating 1 planar graphs of 2000 vertices for training
created dir:  data/training/planar_2000vertices_unweighted
density: 99
Creating 1 planar graphs of 2000 vertices for validation
created dir:  data/validation/planar_2000vertices_unweighted
density: 99
Creating 1 planar graphs of 2000 vertices for training
created dir:  data/training/planar_2000vertices_weighted
density: 99
Creating 1 planar graphs of 2000 vertices for validation
created dir:  data/validation/planar_2000vertices_weighted
density: 99
Creating 1 planar graphs of 1000 vertices for training
created dir:  data/training/planar_1000vertices_unweighted
density: 100
Creating 1 planar graphs of 1000 vertices for validation
created dir:  data/validation/planar_1000vertices_unweighted
density: 100
Creating 1 planar graphs of 800 vertices for training
created dir:  data/training/planar_800vertices_unweighted
density: 99
Creating 1 planar graphs of 800 vertices for validation
created dir:  data/validation/planar_800vertices_unwei

# Torodial 2D

In [40]:
parameters = [
    ([100, 50, 25], [8, 16, 32], False),
    ([100, 80, 50], [20, 25, 40], True),
    ([50, 30, 25], [60, 100, 120], True)]

for heights,widths,weighted in parameters:
    generate_and_save_graphs(distribution='-toroidal_grid_2D', 
                             number_of_graphs=1, set_type='training', 
                            heights=heights,widths=widths, weighted=weighted)
    generate_and_save_graphs(distribution='-toroidal_grid_2D', 
                             number_of_graphs=1, set_type='validation', 
                            heights=heights,widths=widths, weighted=weighted)
    




Creating 1 toroidal_grid_2D graphs of  vertices for training
heights: [100, 50, 25]
widths: [8, 16, 32]
created dir:  data/training/toroidal_grid_2D_800vertices_unweighted
['rudy', '-toroidal_grid_2D', '100', '8']
Creating 1 toroidal_grid_2D graphs of  vertices for validation
heights: [100, 50, 25]
widths: [8, 16, 32]
created dir:  data/validation/toroidal_grid_2D_800vertices_unweighted
['rudy', '-toroidal_grid_2D', '25', '32']
Creating 1 toroidal_grid_2D graphs of  vertices for training
heights: [100, 80, 50]
widths: [20, 25, 40]
created dir:  data/training/toroidal_grid_2D_2000vertices_weighted
['rudy', '-toroidal_grid_2D', '50', '40']
Creating 1 toroidal_grid_2D graphs of  vertices for validation
heights: [100, 80, 50]
widths: [20, 25, 40]
created dir:  data/validation/toroidal_grid_2D_2000vertices_weighted
['rudy', '-toroidal_grid_2D', '100', '20']
Creating 1 toroidal_grid_2D graphs of  vertices for training
heights: [50, 30, 25]
widths: [60, 100, 120]
created dir:  data/training/t

# GSET data download

In [1]:
import wget
import numpy as np

In [4]:
for i in range(1,55):

    dataset_url=f"https://web.stanford.edu/~yyye/yyye/Gset/G{i}"
    wget.download(dataset_url, 'tmp.txt')

    with open("tmp.txt", "r") as file:
        content = file.read()
    
    lines = content.split('\n')
    n = int(lines[0].split(' ')[0])
    graph = np.zeros(shape=(n, n))
    data = [tuple(map(int, line.split())) for line in lines[1:-1]]
    for u, v, w in data:
        graph[u-1, v-1] = w
        graph[v-1, u-1] = w

    save_file_name=f'G{str(i).zfill(2)}'


    if i<=5:
        folder='../data/testing/rnd_graph_800vertices_unweighted/'
    elif i<=10:
        folder='../data/testing/rnd_graph_800vertices_weighted/'
    elif i<=13:
        folder='../data/testing/toroidal_grid_2D_800vertices_weighted'
    elif i<=17:
        folder='../data/testing/planar_800vertices_unweighted'
    elif i<=21:
        folder='../data/testing/planar_800vertices_weighted'
    elif i<=26:
        folder='../data/testing/rnd_graph_2000vertices_unweighted/'
    elif i<=31:
        folder='../data/testing/rnd_graph_2000vertices_weighted/'
    elif i<=34:
        folder='../data/testing/toroidal_grid_2D_2000vertices_weighted'
    elif i<=38:
        folder='../data/testing/planar_2000vertices_unweighted'
    elif i<=42:
        folder='../data/testing/planar_2000vertices_weighted'
    elif i<=47:
        folder='../data/testing/rnd_graph_1000vertices_unweighted/'
    elif i<=50:
        folder='../data/testing/toroidal_grid_2D_3000vertices_unweighted'
    elif i<=54:
        folder='../data/testing/planar_1000vertices_unweighted'
    
    mk_dir(folder)


    save_file_path=os.path.join(folder,save_file_name)
        
    sparse_matrix = csr_matrix(graph)
        
    save_npz(save_file_path, sparse_matrix)

    os.remove('tmp.txt')


created dir:  ../data/testing/rnd_graph_800vertices_unweighted/
dir already exists:  ../data/testing/rnd_graph_800vertices_unweighted/
dir already exists:  ../data/testing/rnd_graph_800vertices_unweighted/
dir already exists:  ../data/testing/rnd_graph_800vertices_unweighted/
dir already exists:  ../data/testing/rnd_graph_800vertices_unweighted/
created dir:  ../data/testing/rnd_graph_800vertices_weighted/
dir already exists:  ../data/testing/rnd_graph_800vertices_weighted/
dir already exists:  ../data/testing/rnd_graph_800vertices_weighted/
dir already exists:  ../data/testing/rnd_graph_800vertices_weighted/
dir already exists:  ../data/testing/rnd_graph_800vertices_weighted/
created dir:  ../data/testing/toroidal_grid_2D_800vertices_weighted
dir already exists:  ../data/testing/toroidal_grid_2D_800vertices_weighted
dir already exists:  ../data/testing/toroidal_grid_2D_800vertices_weighted
created dir:  ../data/testing/planar_800vertices_unweighted
dir already exists:  ../data/testing

In [4]:
OPT= [ 11624,
       11620,
       11622,
       11646,
       11631,
       2178,
       2006,
       2005,
       2054,
       2000,
       564,
       556,
       582,
       3064,
       3050,
       3052,
       3047,
       992,
       906,
       941,
       931,
       13359,
       13344,
       13337,
       13340,
       13328,
       3341,
       3298,
       3405,
       3412,
       3309,
       1410,
       1382,
       1384,
       7684,
       7678,
       7689,
       7687,
       2408,
       2400,
       2405,
       2481,
       6660,
       6650,
       6654,
       6649,
       6657,
       6000,
       6000,
       5880,
       3848,
       3851,
       3850,
       3852
]

In [4]:
folder_names = []

for i in range(1, 55):  # Assuming i ranges from 1 to 54
    if i <= 5:
        folder_names.append('../data/testing/rnd_graph_800vertices_unweighted/')
    elif i <= 10:
        folder_names.append('../data/testing/rnd_graph_800vertices_weighted/')
    elif i <= 13:
        folder_names.append('../data/testing/toroidal_grid_2D_800vertices_weighted')
    elif i <= 17:
        folder_names.append('../data/testing/planar_800vertices_unweighted')
    elif i <= 21:
        folder_names.append('../data/testing/planar_800vertices_weighted')
    elif i <= 26:
        folder_names.append('../data/testing/rnd_graph_2000vertices_unweighted/')
    elif i <= 31:
        folder_names.append('../data/testing/rnd_graph_2000vertices_weighted/')
    elif i <= 34:
        folder_names.append('../data/testing/toroidal_grid_2D_2000vertices_weighted')
    elif i <= 38:
        folder_names.append('../data/testing/planar_2000vertices_unweighted')
    elif i <= 42:
        folder_names.append('../data/testing/planar_2000vertices_weighted')
    elif i <= 47:
        folder_names.append('../data/testing/rnd_graph_1000vertices_unweighted/')
    elif i <= 50:
        folder_names.append('../data/testing/toroidal_grid_2D_3000vertices_unweighted')
    elif i <= 54:
        folder_names.append('../data/testing/planar_1000vertices_unweighted')

print(folder_names)


['../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/toroidal_grid_2D_800vertices_weighted', '../data/testing/toroidal_grid_2D_800vertices_weighted', '../data/testing/toroidal_grid_2D_800vertices_weighted', '../data/testing/planar_800vertices_unweighted', '../data/testing/planar_800vertices_unweighted', '../data/testing/planar_800vertices_unweighted', '../data/testing/planar_800vertices_unweighted', '../data/testing/planar_800vertices_weighted', '../data/testing/planar_800vertices_weighted', '../data/testing/p

In [2]:
folders=['../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_unweighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/rnd_graph_800vertices_weighted/', '../data/testing/toroidal_grid_2D_800vertices_weighted', '../data/testing/toroidal_grid_2D_800vertices_weighted', '../data/testing/toroidal_grid_2D_800vertices_weighted', '../data/testing/planar_800vertices_unweighted', '../data/testing/planar_800vertices_unweighted', '../data/testing/planar_800vertices_unweighted', '../data/testing/planar_800vertices_unweighted', '../data/testing/planar_800vertices_weighted', '../data/testing/planar_800vertices_weighted', '../data/testing/planar_800vertices_weighted', '../data/testing/planar_800vertices_weighted', '../data/testing/rnd_graph_2000vertices_unweighted/', '../data/testing/rnd_graph_2000vertices_unweighted/', '../data/testing/rnd_graph_2000vertices_unweighted/', '../data/testing/rnd_graph_2000vertices_unweighted/', '../data/testing/rnd_graph_2000vertices_unweighted/', '../data/testing/rnd_graph_2000vertices_weighted/', '../data/testing/rnd_graph_2000vertices_weighted/', '../data/testing/rnd_graph_2000vertices_weighted/', '../data/testing/rnd_graph_2000vertices_weighted/', '../data/testing/rnd_graph_2000vertices_weighted/', '../data/testing/toroidal_grid_2D_2000vertices_weighted', '../data/testing/toroidal_grid_2D_2000vertices_weighted', '../data/testing/toroidal_grid_2D_2000vertices_weighted', '../data/testing/planar_2000vertices_unweighted', '../data/testing/planar_2000vertices_unweighted', '../data/testing/planar_2000vertices_unweighted', '../data/testing/planar_2000vertices_unweighted', '../data/testing/planar_2000vertices_weighted', '../data/testing/planar_2000vertices_weighted', '../data/testing/planar_2000vertices_weighted', '../data/testing/planar_2000vertices_weighted', '../data/testing/rnd_graph_1000vertices_unweighted/', '../data/testing/rnd_graph_1000vertices_unweighted/', '../data/testing/rnd_graph_1000vertices_unweighted/', '../data/testing/rnd_graph_1000vertices_unweighted/', '../data/testing/rnd_graph_1000vertices_unweighted/', '../data/testing/toroidal_grid_2D_3000vertices_unweighted', '../data/testing/toroidal_grid_2D_3000vertices_unweighted', '../data/testing/toroidal_grid_2D_3000vertices_unweighted', '../data/testing/planar_1000vertices_unweighted', '../data/testing/planar_1000vertices_unweighted', '../data/testing/planar_1000vertices_unweighted', '../data/testing/planar_1000vertices_unweighted']

In [11]:
for folder in folders:
    instances=os.listdir(folder)
    df={'Instances':[],'OPT':[]}
    instances.sort()
    for instance in instances:
        if instance.endswith('.npz'):
            graph_no=int(instance[1:3])-1
            df['Instances'].append(instance)
            df['OPT'].append(OPT[graph_no])

    # print(instances)
    df=pd.DataFrame(df)
    save_path=os.path.join(folder,'optimal')
    df.to_pickle(save_path)
    # break

In [12]:
df

,Instances,OPT
0,G51.npz,3848
1,G52.npz,3851
2,G53.npz,3850
3,G54.npz,3852
